In [ ]:
!nvidia-smi

In [ ]:
import os,sys,re,glob,random
import pandas as pd
import librosa as lb
import torchaudio
import IPython.display as ipd
import soundfile as sf
import numpy as np
import cv2
import ast
from pathlib import Path

%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
from sklearn import preprocessing

#Deep learning from pytorch
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F

from albumentations.core.transforms_interface import ImageOnlyTransform
from tqdm import tqdm
import joblib

In [ ]:
#timmのdirpathを設定
timm_path = "../input/timm-pytorch-image-models/pytorch-image-models-master"

import sys
sys.path.append(timm_path)
import timm

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
class cfg:
    CLASS_NUM = 21
    period = 5
    sr = 32000
    n_mels = 128
    fmin = 20
    fmax = 16000
    n_fft = 32000//10
    hop_len = 3200//4
    power = 2
    top_db = 80
    
    #timm model name
    model_names = [
        "eca_nfnet_l0",
        "resnest50d",
        "convnext_tiny",
        "eca_nfnet_l0",
        "eca_nfnet_l0",
        "eca_nfnet_l0",
        "eca_nfnet_l0",
        "eca_nfnet_l0",
        "eca_nfnet_l0",
        "eca_nfnet_l0",
        "eca_nfnet_l0",
        "eca_nfnet_l0",
        "eca_nfnet_l0",    
        "eca_nfnet_l0",
        "eca_nfnet_l0",
        "eca_nfnet_l0",
        "eca_nfnet_l0",
        "eca_nfnet_l0", 
        "eca_nfnet_l0",
        "eca_nfnet_l0",
        "eca_nfnet_l0",
        "eca_nfnet_l0",
        "eca_nfnet_l0", 
        "eca_nfnet_l0",
        "eca_nfnet_l0",
        "eca_nfnet_l0",
        "eca_nfnet_l0",
        "eca_nfnet_l0", 
        "eca_nfnet_l0",
        "eca_nfnet_l0",
        "eca_nfnet_l0",
        "eca_nfnet_l0",
        "eca_nfnet_l0", 
        "convnext_tiny",
        "convnext_tiny",
        "convnext_tiny",
        "convnext_tiny",
        "convnext_tiny",
    ]
    
    #weight path
    paths = [
        "../input/bridclef2022dataset/eca_nfnet_l0_p7CV0.881.bin",
        "../input/bridclef2022dataset/resnest50d_CV0.852.bin",     
        "../input/bridclef2022dataset/convnext_tiny_CV0.83.bin",
        "../input/birdclef20225fold/ecanfnetl0_0501/fold0.bin",
        "../input/birdclef20225fold/ecanfnetl0_0501/fold1.bin",
        "../input/birdclef20225fold/ecanfnetl0_0501/fold2.bin",
        "../input/birdclef20225fold/ecanfnetl0_0501/fold3.bin",
        "../input/birdclef20225fold/ecanfnetl0_0501/fold4.bin",
        "../input/birdclef20225fold/ecanfnet0504_reductbias/fold0.bin",
        "../input/birdclef20225fold/ecanfnet0504_reductbias/fold1.bin",
        "../input/birdclef20225fold/ecanfnet0504_reductbias/fold2.bin",
        "../input/birdclef20225fold/ecanfnet0504_reductbias/fold3.bin",
        "../input/birdclef20225fold/ecanfnet0504_reductbias/fold4.bin",
        "../input/birdclef20225fold/ecanfnetl0_CV0.80/fold0.bin",
        "../input/birdclef20225fold/ecanfnetl0_CV0.80/fold1.bin",
        "../input/birdclef20225fold/ecanfnetl0_CV0.80/fold2.bin",
        "../input/birdclef20225fold/ecanfnetl0_CV0.80/fold3.bin",
        "../input/birdclef20225fold/ecanfnetl0_CV0.80/fold4.bin",
        "../input/birdclef20225fold/efl0img_0515_fulllabeling/fold0.bin",
        "../input/birdclef20225fold/efl0img_0515_fulllabeling/fold1.bin",
        "../input/birdclef20225fold/efl0img_0515_fulllabeling/fold2.bin",
        "../input/birdclef20225fold/efl0img_0515_fulllabeling/fold3.bin",
        "../input/birdclef20225fold/efl0img_0515_fulllabeling/fold4.bin",
        "../input/birdclef20225fold/ecanfnetl0502_mixup2.0/fold0.bin",
        "../input/birdclef20225fold/ecanfnetl0502_mixup2.0/fold1.bin",
        "../input/birdclef20225fold/ecanfnetl0502_mixup2.0/fold2.bin",
        "../input/birdclef20225fold/ecanfnetl0502_mixup2.0/fold3.bin",
        "../input/birdclef20225fold/ecanfnetl0502_mixup2.0/fold4.bin",
        "../input/birdclef20225fold/ecanfnetl0_0431_v2/fold0.bin",
        "../input/birdclef20225fold/ecanfnetl0_0431_v2/fold1.bin",
        "../input/birdclef20225fold/ecanfnetl0_0431_v2/fold2.bin",
        "../input/birdclef20225fold/ecanfnetl0_0431_v2/fold3.bin",
        "../input/birdclef20225fold/ecanfnetl0_0431_v2/fold4.bin",
        "../input/birdclef20225fold/convnext_0502/fold0.bin",
        "../input/birdclef20225fold/convnext_0502/fold1.bin",
        "../input/birdclef20225fold/convnext_0502/fold2.bin",
        "../input/birdclef20225fold/convnext_0502/fold3.bin",
        "../input/birdclef20225fold/convnext_0502/fold4.bin",
    ]
    
    region_paths = [
        "../input/birdclef20225fold/ecanfnetl0_0511_21+region/model_0_0.886.bin",
        "../input/birdclef20225fold/ecanfnetl0_0511_21+region/model_1_0.873.bin",
        "../input/birdclef20225fold/ecanfnetl0_0511_21+region/model_2_0.849.bin",
        "../input/birdclef20225fold/ecanfnetl0_0511_21+region/model_3_0.872.bin",
        "../input/birdclef20225fold/ecanfnetl0_0511_21+region/model_4_0.824.bin"
    ]
    
    region_model_names = [
        "eca_nfnet_l0",
        "eca_nfnet_l0",
        "eca_nfnet_l0",
        "eca_nfnet_l0",
        "eca_nfnet_l0",
    ]
        
    
    scored_birds = [
        "akiapo",
        "aniani", 
        "apapan", 
        "barpet", 
        "crehon",#5 ラベル少ないNo.2
        "elepai", 
        "ercfra",#7 ラベル少ないNo.6
        "hawama", 
        "hawcre", 
        "hawgoo",#10 ラベル少ないNo.7
        "hawhaw",#11 ラベル少ないNo.4
        "hawpet1",#12 ラベルすくないNo.5
        "houfin",#13 ラベルトップ No.2 
        "iiwi", 
        "jabwar", 
        "maupar",#16 絶滅危惧種No.1
        "omao", 
        "puaioh",#18 絶滅危惧種No.3
        "skylar",#13 ラベルトップ No.1
        "warwhe1", 
        "yefcan"
    ]


In [ ]:
class WaveformDataset:
    def __init__(self,
                 df: pd.DataFrame,
                 period,
                 sr, 
                 n_mels, 
                 fmin, 
                 fmax,
                 ):
        
        self.df = df
        
        #make Melspectrum
        self.sr = sr
        self.period = period
        self.n_mels = n_mels
        self.fmin = fmin
        self.fmax = fmax
        kwargs = {}
        kwargs["n_fft"] = kwargs.get("n_fft", self.sr//10)
        kwargs["hop_length"] = kwargs.get("hop_length", self.sr//(10*4))
        self.kwargs = kwargs
    
    def make_melspec(self, y):

        melspec = librosa.feature.melspectrogram(
            y=y, 
            sr=self.sr, 
            n_mels=self.n_mels, 
            fmin=self.fmin, 
            fmax=self.fmax, 
            **self.kwargs,
        )

        melspec = librosa.power_to_db(melspec).astype(np.float32)
        return melspec
    
    def mono_to_color(self, X, eps=1e-6, mean=None, std=None):
        mean = mean or X.mean()
        std = std or X.std()
        X = (X - mean) / (std + eps)

        _min, _max = X.min(), X.max()

        if (_max - _min) > eps:
            V = np.clip(X, _min, _max)
            V = 255 * (V - _min) / (_max - _min)
            V = V.astype(np.uint8)
        else:
            V = np.zeros_like(X, dtype=np.uint8)

        return V

    def crop_or_pad(self, y, length, is_train=False, start=None):
        if len(y) < length:
            y = np.concatenate([y, np.zeros(length - len(y))])

            n_repeats = length // len(y)
            epsilon = length % len(y)

            y = np.concatenate([y]*n_repeats + [y[:epsilon]])

        elif len(y) > length:
            if not is_train:
                start = start or 0
            else:
                start = start or np.random.randint(len(y) - length)

            y = y[start:start + length]

        return y
    
    def audio_to_image(self, audio):
        melspec = self.make_melspec(audio) 
        image = self.mono_to_color(melspec)
        return image

    def __call__(self, path):        
        #データ読み込み
        data, sr = librosa.load(path, sr=self.sr)
        
        #test datasetの最大長
        max_sec = 60
        
        #予測フレーム
        pred_sec = 7
        
        #データを5秒間隔でかつ7秒幅を取って区切る
        datas = [data[int(max(0, i-1) * sr):int(min(max_sec, i+6) * sr)] for i in range(0, max_sec, self.period)]
        
        #端は1秒短くなるので埋める
        datas[0] = self.crop_or_pad(datas[0] , length=sr*pred_sec)
        datas[-1] = self.crop_or_pad(datas[-1] , length=sr*pred_sec)
        
        #データをメル周波数によって画像化
        images = [self.audio_to_image(audio) for audio in datas]
        images = np.stack(images)
        
        #保存
        filename = path.split("/")[-1]
        path = f"/kaggle/audio_images/{filename}.npy"
        np.save(path, images)

In [ ]:
def get_audios_as_images(paths):
    pool = joblib.Parallel(2)
    
    converter = WaveformDataset(paths,
                             period=cfg.period,
                             sr=cfg.sr,
                             n_mels=cfg.n_mels,
                             fmin = cfg.fmin,
                             fmax = cfg.fmax,
                            )
    mapper = joblib.delayed(converter)
    tasks = [mapper(path) for path in tqdm(paths)]
    pool(tqdm(tasks))

In [ ]:
!mkdir /kaggle/audio_images
all_audio_paths = glob.glob("../input/birdclef-2022/test_soundscapes/*.ogg")
get_audios_as_images(all_audio_paths)
!ls /kaggle/audio_images/

In [ ]:
class WaveformDataset(Dataset):
    def __init__(self,
                 paths):
        self.paths = paths
        
    @staticmethod
    def normalize(image):
        image = image[:,None,:,:].astype("float32", copy=False) / 255.0
        return image

    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, idx):
        path = self.paths[idx]
        image = np.load(path)
        image = self.normalize(image)
        file_id = path.split("/")[-1].replace(".ogg.npy","")
        
        return image, file_id

In [ ]:
class Model(nn.Module):
    def __init__(self,name,pretrained=False):
        super(Model, self).__init__()
        self.model = timm.create_model(name,pretrained=pretrained, in_chans=1)
        self.model.reset_classifier(num_classes=0) 
        in_features = self.model.num_features
        self.fc = nn.Linear(in_features, cfg.CLASS_NUM)

    def forward(self, x):
        x = self.model(x)
        x = self.fc(x)
        return x
    
class regionModel(nn.Module):
    def __init__(self,name,pretrained=False):
        super(regionModel, self).__init__()
        self.model = timm.create_model(name,pretrained=pretrained, in_chans=1)
        self.model.reset_classifier(num_classes=0) 
        in_features = self.model.num_features
        self.fc = nn.Linear(in_features, cfg.CLASS_NUM+1)
        
    def forward(self, x):
        x = self.model(x)
        x = self.fc(x)
        return x[:,:21], x[:,21]

In [ ]:
models = []
for name, path in zip(cfg.model_names,cfg.paths):
    print(path)
    model = Model(name=name).to(device)
    model.load_state_dict(torch.load(path))
    model.eval()
    models.append(model)

In [ ]:
"""
region_model = []
for name, path in zip(cfg.region_model_names,cfg.region_paths):
    model = regionModel(name=name).to(device)
    model.load_state_dict(torch.load(path),strict=False)
    model.eval()
    region_model.append(model)
"""    
paths = glob.glob("/kaggle/audio_images/*.npy")
paths

In [ ]:
weights = [
    7,7,7,#最初の3つは大きめ
    1,1,1,1,1,#ecanfnetl0_0501
    0.8,0.8,0.8,0.8,0.8,#ecanfnet0504_reductbias
    1,1,1,1,1,#ecanfnetl0_CV0.80
    0.3,0.3,0.3,0.3,0.3,#efl0img_0515_fulllabeling
    0.8,0.8,0.8,0.8,0.8,#ecanfnetl0502_mixup2.0
    0.7,0.7,0.7,0.7,0.7,#ecanfnetl0_0431_v2
    0.5,0.5,0.5,0.5,0.5,#convnext_0502
]

In [ ]:
dataset = WaveformDataset(paths)
proba_arr = []
region_arr = []
file_ids = []

pred_list = [None for _ in range(len(models))]

for n, (images, file_id) in tqdm(enumerate(dataset), total=len(dataset)):
    with torch.no_grad():
        images = torch.tensor(images, dtype=torch.float).to(device)
        pred_l = None
        
        for i,model in enumerate(models):
            if i <= 2:
                pred = model(images).sigmoid().detach().cpu().numpy()
            else:
                pred1 = model(images[:,:,:,0:201]).sigmoid().detach().cpu().numpy()
                pred2 = model(images[:,:,:,40:241]).sigmoid().detach().cpu().numpy()
                pred3 = model(images[:,:,:,80:281]).sigmoid().detach().cpu().numpy()
                pred = 0.25*pred1 + 0.5*pred2 + 0.25*pred3
            if i==0:
                preds =  pred* weights[i]/sum(weights)
                
            else:
                preds += pred* weights[i]/sum(weights)
                
            if n==0:
                pred_list[i] = pred
            else:
                pred_list[i] = np.concatenate([pred_list[i], pred])
        
        proba_arr.append(preds)
            
        """
        for i,model in enumerate(region_model):
            #pred1,  = model(images[:,:,:,0:201]).sigmoid().detach().cpu().numpy()
            region = model(images[:,:,:,40:241])[1].sigmoid().detach().cpu().numpy()
            #pred3, _ = model(images[:,:,:,80:281]).sigmoid().detach().cpu().numpy()
            #pred = 0.25*pred1 + 0.5*pred2 + 0.25*pred3
            if i==0:
                preds = region/len(region_model)
            else:
                preds += region/len(region_model)
        
        region_arr.append(preds)
        """
        file_ids.append(file_id)

# Voting Re-scoring Post-Processing

In [ ]:
judge_percentile1 = 0.90
judge_percentile2 = 0.80
judge_percentile3 = 0.7

judge1_birds = [ 
        "apapan", 
        "barpet", 
        "hawcre", 
        "houfin",#13 ラベルトップ No.2 
        "iiwi", 
        "jabwar", 
        "skylar",#13 ラベルトップ No.1
    ]

judge2_birds = [
        "akiapo",
        "aniani", 
        "elepai", 
        "hawama", 
        "omao", 
        "warwhe1", 
        "yefcan"
    ]

judge3_birds = [ 
        "crehon",#5 ラベル少ないNo.2
        "ercfra",#7 ラベル少ないNo.6
        "hawgoo",#10 ラベル少ないNo.7
        "hawhaw",#11 ラベル少ないNo.4
        "hawpet1",#12 ラベルすくないNo.5  
        "maupar",#16 絶滅危惧種No.1
        "puaioh",#18 絶滅危惧種No.3
    ]

In [ ]:
score_df = np.zeros(pred_list[0].shape)
for pred in pred_list:
    for i,specie in enumerate(cfg.scored_birds):
        pred_per_s = pred[:,i]
        if specie in judge1_birds:
            thre = np.percentile(pred_per_s, judge_percentile1)
        elif specie in judge2_birds:
            thre = np.percentile(pred_per_s, judge_percentile2)
        elif specie in judge3_birds:
            thre = np.percentile(pred_per_s, judge_percentile3)
        score = np.where(pred_per_s>thre, 1, 0)
        score_df[:,i] += score

アンサンブルするモデルが指定した閾値を超えているかどうかをvotingする。
そのvotingに従って、予測確率に対する重みを最大2倍まで高める

In [ ]:
weight_df = score_df.copy()
weight_df = weight_df/len(models)+1
weight_df

In [ ]:
#weight_df = np.concatenate([weight_df, weight_df, weight_df[:10]])
#proba_arr = proba_arr + proba_arr + [proba_arr[0][:10]]

In [ ]:
num = 0
for i, prob in enumerate(proba_arr):
    extract_len = len(prob)
    weight = weight_df[num:num+extract_len]
    proba_arr[i] = prob * weight
    num += extract_len

# Make Submission

In [ ]:
#judge_percentile1 = 85
judge_percentile2 = 70
#judge_percentile2 = 50


"""judge1_birds = [ 
        "apapan", 
        "aniani",
        "hawama", 
        "hawcre", 
        "houfin",#13 ラベルトップ No.2 
        "jabwar", 
        "omao",
        "skylar",#13 ラベルトップ No.1
    ]
judge1_class = [i for i,s in enumerate(cfg.scored_birds) if s in judge1_birds]
other_class = [i for i in range(len(cfg.scored_birds)) if not i in judge1_class]"""
judge2_birds = [
        "barpet", 
        "crehon",#5 ラベル少ないNo.2
        "elepai", 
        "ercfra",#7 ラベル少ないNo.6
        "hawhaw",#11 ラベル少ないNo.4
        "hawpet1",#12 ラベルすくないNo.5
        "maupar",#16 絶滅危惧種No.1
    ]
judge2_class = [i for i,s in enumerate(cfg.scored_birds) if s in judge2_birds]
other_class = [i for i in range(len(cfg.scored_birds)) if not i in judge2_birds]

In [ ]:
#LBPBdataの予測平均をthreshouldとして予測
thresholds = np.array([np.array(preds.mean(0)) for preds in proba_arr])
threshold80 = np.percentile(thresholds,80,axis=0)
concat_proba_arr = np.concatenate(proba_arr)
#threshold_j1 = np.percentile(concat_proba_arr,judge_percentile1,axis=0)
threshold_j2 = np.percentile(concat_proba_arr,judge_percentile2,axis=0)
print(threshold80)
threshold = np.zeros(threshold80.shape)
threshold[judge2_class] = threshold_j2[judge2_class]
threshold[other_class] = threshold80[other_class]
print(threshold)
#threshold = thresholds.mean(0)

#レアラベルの場合は半分FP許容
audio_level_pred_thre = np.percentile(thresholds,70,axis=0)

#中間層以外はaudio_levelはしない
audio_level_pred_thre[[0,1,3,4,5,6,9,10,11,12,15,17,18]] = 1.0

display(threshold,audio_level_pred_thre)

In [ ]:
prediction_dict = {}
for preds, file_id in zip(proba_arr,file_ids): 
    
    #１度鳴いてる鳥は全部鳴いてる確率が高い
    index = np.argwhere(preds.mean(0) > np.array(audio_level_pred_thre))
    if len(index) != 0:
        preds[:,index] = 1.0
        
    for idx in range(12):
        last_time = int((idx+1)*5)
        
        #どうやら60秒未満のものがあるらしい[index 3000あたり？]
        try:
            probas = preds[idx]
 
        except:
            probas = probas
        
        row_id = file_id + "_" + str(last_time)   
        labels = np.argwhere(probas >= threshold).reshape(-1).tolist()

        if len(labels) == 0:
            prediction_dict[str(row_id)] = "nocall"
        else:
            labels_str_list = list(map(lambda x: cfg.scored_birds[x], labels))
            label_string = " ".join(labels_str_list)
            prediction_dict[str(row_id)] = label_string

In [ ]:
sample_submission = pd.read_csv('../input/birdclef-2022/sample_submission.csv')
sample_submission["target"] = True
sample_submission

In [ ]:
for i in range(len(sample_submission)):
    #soundscape_1000170626_akiapo_5 (sample)
    sample = sample_submission.row_id[i]
    #soundscape_1000170626_5 (key)
    key = sample.split("_")[0] + "_" + sample.split("_")[1] + "_" + sample.split("_")[3]
    #akiapo (target_bird)
    target_bird = sample.split("_")[2]
    print(key, target_bird)
    # soundscape_1000170626_5 in {soundscape_1000170626_5:"nocall",'soundscape_453028782_5': 'nocall',....}
    if key in prediction_dict:
        #"nocall" (prediction_dicts[key]) → False
        sample_submission.iat[i, 1] = (target_bird in prediction_dict[key])
sample_submission.to_csv("submission.csv", index=False)

In [ ]:
sample_submission